In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Load dataset
file_path = 'dataset_preprocess_fix.csv'  # Ganti dengan path dataset Anda
df = pd.read_csv(file_path)

df.head(5)

,Unnamed: 0,No,Text,Label,tokens,tokens_fdist,normalized,tokens_stemmed,joined_tokens
0,0,1,baru saja kenalan sama orang baru di kafe dan ...,1,"['baru', 'saja', 'kenalan', 'sama', 'orang', '...","['kenalan', 'orang', 'kafe', 'seru', 'banget',...","['baru', 'saja', 'kenalan', 'sama', 'orang', '...","['baru', 'saja', 'kenal', 'sama', 'orang', 'ba...",baru saja kenal sama orang baru di kafe dan se...
1,1,2,gak bisa diem di rumah nih pengen banget jalan...,1,"['gak', 'bisa', 'diem', 'di', 'rumah', 'nih', ...","['diem', 'rumah', 'pengen', 'banget', 'jalanja...","['tidak', 'bisa', 'diem', 'di', 'rumah', 'nih'...","['tidak', 'bisa', 'diem', 'di', 'rumah', 'nih'...",tidak bisa diem di rumah nih ken banget jalanj...
2,2,3,ngecash energi dengan ketemu orang kantor nih ...,1,"['ngecash', 'energi', 'dengan', 'ketemu', 'ora...","['ngecash', 'energi', 'ketemu', 'orang', 'kant...","['ngecash', 'energi', 'dengan', 'ketemu', 'ora...","['ngecash', 'energi', 'dengan', 'ketemu', 'ora...",ngecash energi dengan ketemu orang kantor nih ...
3,3,4,hari ini sibuk banget tapi seneng karena bisa ...,1,"['hari', 'ini', 'sibuk', 'banget', 'tapi', 'se...","['sibuk', 'banget', 'seneng', 'ketemu', 'orang...","['hari', 'ini', 'sibuk', 'banget', 'tetapi', '...","['hari', 'ini', 'sibuk', 'banget', 'tetapi', '...",hari ini sibuk banget tetapi neng karena bisa ...
4,4,5,pesta di rumah tetangga bikin hari jadi gaul b...,1,"['pesta', 'di', 'rumah', 'tetangga', 'bikin', ...","['pesta', 'rumah', 'tetangga', 'gaul', 'banget...","['pesta', 'di', 'rumah', 'tetangga', 'bikin', ...","['pesta', 'di', 'rumah', 'tetangga', 'bikin', ...",pesta di rumah tetangga bikin hari jadi gaul b...


In [3]:
df['Label'] = df['Label'] - 1

In [4]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['joined_tokens'], df['Label'], test_size=0.2, random_state=42)

In [5]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing the data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)


C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# Convert the tokenized data into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)

In [7]:
# Determine the number of unique labels
num_labels = len(df['Label'].unique())

# Load the BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [9]:
# Train the model
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=10,
                    callbacks=[early_stopping])

Epoch 1/10
81/81 [==============================] - 714s 8s/step - loss: 2.1032 - accuracy: 0.2036 - val_loss: 1.6094 - val_accuracy: 0.1605
Epoch 2/10
81/81 [==============================] - 689s 8s/step - loss: 1.6094 - accuracy: 0.2020 - val_loss: 1.6094 - val_accuracy: 0.1605
Epoch 3/10
81/81 [==============================] - 570s 7s/step - loss: 1.6094 - accuracy: 0.1935 - val_loss: 1.6094 - val_accuracy: 0.1605
Epoch 4/10
81/81 [==============================] - 563s 7s/step - loss: 1.6094 - accuracy: 0.2090 - val_loss: 1.6094 - val_accuracy: 0.1605
